In [1]:
import pandas as pd 
import process
import numpy as np 
# Jerome path : r'C:\Users\33640\OneDrive\Documents\GitHub\Portfolio_clustering_project\Data\DataBase.csv'
# Nail path : '/Users/khelifanail/Documents/GitHub/Portfolio_clustering_project/Data/DataBase.csv'
df = pd.read_csv(r'/Users/khelifanail/Documents/GitHub/Portfolio_clustering_project/Data/DataBase.csv')

df.set_index('ticker', inplace=True)

df.columns = pd.to_datetime(df.columns.str[1:], format='%Y%m%d').strftime('%d/%m/%Y')

df_cleaned = df.fillna(0) # Utilisez la méthode fillna(0) pour remplacer les NaN par 0

In [94]:
import warnings 
warnings.filterwarnings('ignore')
lookback_window = [0, 50]
number_of_clusters = 38

def cluster_composition_and_centroid(self):
## cluster composition and centroids

result = dict(zip(list(self.correlation_matrix.columns), self.apply_SPONGE())) ## composition

df_cleaned['Cluster'] = df_cleaned.index.map(result)
centroid_returns = df_cleaned.groupby('Cluster').mean() ## centroids 

df_cleaned = df_cleaned.transpose()
centroid_returns = centroid_returns.transpose()

## constituent_weights
centred_returns = df_cleaned.copy()
sigma = 0.01

constituent_weights = pd.DataFrame(index=['Weight'], columns=centred_returns.columns)
total_weight = pd.DataFrame(index=['Total weight'], columns=[i for i in range(number_of_clusters)], data=np.zeros((1, number_of_clusters)))

## we first compute the difference between the cluster centroid return and the cluster ticker return
for ticker in centred_returns.columns:
    centred_returns[ticker][:-1] = centred_returns[ticker][:-1] - centroid_returns[int(centred_returns[ticker]['Cluster'])]

## we use this difference to compute the distance between each asset and its cluster centroid return 
for ticker in centred_returns.columns:
    constituent_weights[ticker] = np.exp(sigma*((np.linalg.norm(centred_returns[ticker][:-1]))**2)/2)
    total_weight[int(centred_returns[ticker]['Cluster'])]['Total weight'] += np.exp(sigma*((np.linalg.norm(centred_returns[ticker][:-1]))**2)/2)

## we normalize the weights
for ticker in centred_returns.columns:
    constituent_weights[ticker] = constituent_weights[ticker]['Weight']/total_weight[int(centred_returns[ticker]['Cluster'])]['Total weight']

## check whether the weights equal to 0 within each cluster: 
# constituent_weights[[ticker for ticker in centred_returns.columns if centred_returns[ticker]['Cluster']  == 1.0]].sum(axis=1)
    
## cluster returns 
    
cluster_returns = pd.DataFrame(index=df_cleaned.index[:-1], columns=np.arange(number_of_clusters), data=np.zeros((df_cleaned.shape[0] - 1, number_of_clusters)))

for ticker in df_cleaned.columns:
    cluster_returns[int(df_cleaned[ticker]['Cluster'])] = cluster_returns[int(df_cleaned[ticker]['Cluster'])] + constituent_weights[ticker]['Weight'] * df_cleaned[ticker][:-1]

IndentationError: expected an indented block after function definition on line 6 (2352418720.py, line 9)

In [3]:
from pypfopt.efficient_frontier import EfficientFrontier
import process
from PyFolioCC import PyFolioCC 

historical_data = df_cleaned
lookback_window = [0, 50]
evaluation_window = 1
number_of_clusters = 38
clustering_method = 'SPONGE'
sigma = 0.1
eta = 0.1

corr_matrix = process.correlation_matrix(lookback_window=lookback_window, df_cleaned=df_cleaned)
cluster_comp = process.cluster_composition_and_centroid(df_cleaned=df_cleaned, correlation_matrix=corr_matrix, number_of_clusters=number_of_clusters, lookback_window=lookback_window, clustering_method=clustering_method)
constituent_weights = process.constituent_weights(df_cleaned=df_cleaned, cluster_composition=cluster_comp, sigma=sigma, lookback_window=lookback_window)
cluster_return_res = process.cluster_return(constituent_weights=constituent_weights, df_cleaned=df_cleaned, lookback_window=lookback_window)
# markowitz_weights_res = process.markowitz_weights(cluster_return_res=cluster_return, constituent_weights=constituent_weights, df_cleaned=df_cleaned, lookback_window=lookback_window, evaluation_window=evaluation_window, eta=eta)

In [12]:
cov_matrix = cluster_return_res.cov()

cov_matrix.fillna(0.)

## on construit le vecteur d'expected return du cluster (252 jours de trading par an, on passe de rendements journaliers à rendements annualisés)

cluster_target_return = process.cluster_return(constituent_weights=constituent_weights, df_cleaned=df_cleaned, lookback_window=[lookback_window[1], lookback_window[1]+evaluation_window]).mean()

expected_returns = process.noised_array(y=cluster_target_return, eta=eta)

ef = EfficientFrontier(expected_returns=expected_returns, cov_matrix=cov_matrix, weight_bounds=(0, 1))

ef.efficient_return(target_return=expected_returns.mean())

markowitz_weights = ef.clean_weights()

In [13]:
markowitz_weights

OrderedDict([('cluster 1', 0.0),
             ('cluster 2', 0.07679),
             ('cluster 3', 0.01276),
             ('cluster 4', 0.0),
             ('cluster 5', 0.0),
             ('cluster 6', 0.0),
             ('cluster 7', 0.00707),
             ('cluster 8', 0.0),
             ('cluster 9', 0.0),
             ('cluster 10', 0.02374),
             ('cluster 11', 0.66794),
             ('cluster 12', 0.0),
             ('cluster 13', 0.02627),
             ('cluster 14', 0.03291),
             ('cluster 15', 0.00318),
             ('cluster 16', 0.0),
             ('cluster 17', 0.0),
             ('cluster 18', 0.01033),
             ('cluster 19', 0.00849),
             ('cluster 20', 0.01538),
             ('cluster 21', 0.0),
             ('cluster 22', 0.0),
             ('cluster 23', 0.0003),
             ('cluster 24', 0.03151),
             ('cluster 25', 0.0),
             ('cluster 26', 0.0),
             ('cluster 27', 0.0),
             ('cluster 28', 0.06241),
  

In [31]:
cluster_returns = pd.DataFrame(index = df_cleaned.columns[lookback_window[0]:lookback_window[1]], columns= [f'cluster {i}' for i in range(1, number_of_clusters + 1)], data = np.zeros((len(df_cleaned.columns[lookback_window[0]:lookback_window[1]]), len(constituent_weights) + 1)))

for cluster in constituent_weights.keys():
    for ticker, weight in constituent_weights[cluster].items():
        cluster_returns[cluster] = cluster_returns[cluster] + df_cleaned.transpose()[ticker][lookback_window[0]:lookback_window[1]]*weight

In [60]:
corr_matrix = process.correlation_matrix(lookback_window=lookback_window, df_cleaned=df_cleaned)
cluster_comp = process.cluster_composition_and_centroid(df_cleaned=df_cleaned, correlation_matrix=corr_matrix, number_of_clusters=number_of_clusters, lookback_window=lookback_window, clustering_method=clustering_method)
constituent_weights = process.constituent_weights(df_cleaned=df_cleaned, cluster_composition=cluster_comp, sigma=sigma, lookback_window=lookback_window)
cluster_return = process.cluster_return(constituent_weights=constituent_weights, df_cleaned=df_cleaned, lookback_window=lookback_window)
markowitz_weights_res = process.markowitz_weights(cluster_return_res=cluster_return, constituent_weights=constituent_weights, df_cleaned=df_cleaned, lookback_window=lookback_window, evaluation_window=evaluation_window, eta=eta)
final_weights = process.final_weights(markowitz_weights=markowitz_weights_res, constituent_weights=constituent_weights)
print(corr_matrix.shape, len(cluster_comp), len(constituent_weights), len(cluster_return), len(markowitz_weights_res), len(final_weights))


(695, 695) 37 37 37 37 692


In [10]:
cluster_returns = pd.DataFrame(index = [f'cluster {i}' for i in range(1, len(constituent_weights) + 1)], columns = df_cleaned.columns[lookback_window[0]:lookback_window[1]])
for keys, value in constituent_weights['cluster 1'].items():
    print(keys, value)

AVA 3.921716041425027e-05
BKH 0.110883916406304
BXS 0.2035512296849513
DRQ 0.04082643926897566
INCY 1.0230500389790201e-11
LNT 0.1654632187161882
MTZ 0.14365464588183466
NI 0.05214005001251528
OKE 0.2394986472578898
PKI 0.01829458705579904
RMD 0.02564804854489745
